# 서포트 벡터 머신 (Support Vector Machine, SVM)

**학습 목표:**
- 분류(Classification) 문제에서 강력한 성능을 보이는 SVM의 기본 원리를 이해합니다.
- 클래스 간의 경계(margin)를 최대화하는 최적의 결정 경계(hyperplane)를 찾는 과정을 학습합니다.
- 데이터의 스케일링이 SVM 성능에 미치는 중요성을 확인합니다.
- SVM의 핵심 요소인 **커널(Kernel)** 트릭(linear, rbf)의 차이를 이해하고, 하이퍼파라미터(`C`, `gamma`) 튜닝을 통해 모델 성능을 최적화합니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

### (1) 데이터 준비 및 탐색
scikit-learn에 내장된 유방암 진단 데이터셋을 사용합니다. 각 종양의 특징을 기반으로 악성(malignant)과 양성(benign)을 분류합니다.

In [ ]:
cancer = load_breast_cancer()
X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = pd.Series(cancer.target, name='target')

print('Data Shape:', X.shape)
print('Target Distribution:\n', y.value_counts())
X.head()

### (2) 데이터 스케일링의 중요성
SVM은 데이터 포인트 간의 거리를 기반으로 동작하므로, 특성들의 스케일이 다르면 특정 특성에 과도하게 영향을 받을 수 있습니다. `StandardScaler`를 사용하여 모든 특성의 스케일을 표준화합니다.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 스케일링 전 모델 학습
svm_unscaled = SVC(kernel='linear', random_state=42)
svm_unscaled.fit(X_train, y_train)
pred_unscaled = svm_unscaled.predict(X_test)
acc_unscaled = accuracy_score(y_test, pred_unscaled)
print(f"Accuracy (before scaling): {acc_unscaled:.4f}")

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 스케일링 후 모델 학습
svm_scaled = SVC(kernel='linear', random_state=42)
svm_scaled.fit(X_train_scaled, y_train)
pred_scaled = svm_scaled.predict(X_test_scaled)
acc_scaled = accuracy_score(y_test, pred_scaled)
print(f"Accuracy (after scaling):  {acc_scaled:.4f}")

### (3) 커널(Kernel) 트릭 이해하기
- **Linear Kernel**: 선형적으로 구분 가능한 데이터에 적합합니다.
- **RBF (Radial Basis Function) Kernel**: 비선형 데이터에 효과적이며, 데이터를 고차원 공간으로 매핑하여 선형 분리를 가능하게 합니다. 기본 커널로 많이 사용됩니다.

In [ ]:
# RBF 커널 모델 학습
svm_rbf = SVC(kernel='rbf', random_state=42)
svm_rbf.fit(X_train_scaled, y_train)
pred_rbf = svm_rbf.predict(X_test_scaled)
acc_rbf = accuracy_score(y_test, pred_rbf)

print(f"Linear Kernel Accuracy: {acc_scaled:.4f}")
print(f"RBF Kernel Accuracy:    {acc_rbf:.4f}")

### (4) 하이퍼파라미터 튜닝 (GridSearchCV)
모델 성능에 큰 영향을 미치는 주요 하이퍼파라미터를 최적화합니다.
- `C`: 규제 매개변수. 마진 오류와 분류 오류 사이의 트레이드오프를 결정합니다. (작으면 마진이 넓어지고, 크면 마진이 좁아짐)
- `gamma`: RBF 커널의 영향력. 데이터 포인트 하나가 결정 경계에 미치는 영향 범위를 결정합니다. (작으면 영향이 넓고, 크면 영향이 좁아짐)

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

grid_search = GridSearchCV(SVC(random_state=42), param_grid, cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

print("\n--- Grid Search Results ---")
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# 최적 모델로 평가
best_svm = grid_search.best_estimator_
pred_best = best_svm.predict(X_test_scaled)
acc_best = accuracy_score(y_test, pred_best)

print(f"\nTest set accuracy with best parameters: {acc_best:.4f}")

# Confusion Matrix 시각화
cm = confusion_matrix(y_test, pred_best)
plt.figure(figsize=(7, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=cancer.target_names, yticklabels=cancer.target_names)
plt.title('Confusion Matrix for Best SVM')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

print("\n--- Final Classification Report ---")
print(classification_report(y_test, pred_best, target_names=cancer.target_names))